<a href="https://colab.research.google.com/github/CENFARG/ocr2text2table/blob/main/OCR_simple_con_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Con esta linea le decimos al al compilador (programa que ejecuta el codigo) que instale las herramientas que necesitamos

In [87]:
# Instalar dependencias necesarias
!apt-get install -y poppler-utils tesseract-ocr tesseract-ocr-spa
!pip install pytesseract pdf2image opencv-python pandas openai tabula-py markdownify python-docx xlsxwriter

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-spa is already the newest version (1:4.00~git30-7274cfa-1.1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


Con esta linea instalamos las dependecias previamente instaladas

In [88]:
# importamos las dependencias instaladas
import pytesseract
import cv2
import numpy as np
import pandas as pd
import openai
import os
from pdf2image import convert_from_path
from google.colab import files
import tabula
from PIL import Image
from markdownify import markdownify as md
from docx import Document

Configuramos la herramienta de OCR Tesseract en idioma Español o el que tiene el PDF/IMAGEN

In [100]:
# Configurar Tesseract para usar el idioma español
os.environ["TESSDATA_PREFIX"] = "/usr/share/tesseract-ocr/4.00/tessdata/"

Crear las carpetas de trabajo de los archivos de subida y descargas

In [99]:
# Crear carpetas de trabajo
os.makedirs("upload_files", exist_ok=True)
os.makedirs("output_files", exist_ok=True)

Definimos la funcion para subir archivos pdf o imagenes de hasta 20Mb

In [98]:
def get_unique_filename(folder, filename):
    """Genera un nombre de archivo único si ya existe en la carpeta."""
    base, ext = os.path.splitext(filename)
    counter = 1
    new_filename = filename
    while os.path.exists(os.path.join(folder, new_filename)):
        new_filename = f"{base}_{counter}{ext}"
        counter += 1
    return new_filename

def upload_file():
    """Permite subir un archivo (PDF o imagen) de hasta 20MB"""
    uploaded = files.upload()
    for filename in uploaded.keys():
        file_size = os.stat(filename).st_size / (1024 * 1024)  # Convertir a MB
        if file_size > 20:
            print("El archivo excede el límite de 20MB.")
            return None
        new_path = os.path.join("upload_files", filename)
        os.rename(filename, new_path)
        return new_path

Definimos una funcion que verifica la calidad del PDF o imagen para hacer OCR

In [101]:
def evaluate_pdf_quality(pdf_path):
    """Evalúa la calidad del PDF para OCR (0-100%)"""
    images = convert_from_path(pdf_path)
    total_score = 0
    for img in images:
        img = np.array(img)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        blur = cv2.Laplacian(gray, cv2.CV_64F).var()
        total_score += min(100, max(0, blur / 500 * 100))  # Normalizar
    return total_score / len(images)

Definimos la funcion de extracción de texto (OCR)

In [102]:
def extract_text_from_pdf(pdf_path):
    """Extrae texto de un PDF usando OCR y mantiene formato"""
    images = convert_from_path(pdf_path)
    text = ""
    for img in images:
        text += pytesseract.image_to_string(img, lang='spa', config='--oem 3 --psm 6') + "\n"
    return text

def extract_text_from_image(image_path):
    """Extrae texto de una imagen usando OCR."""
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img, lang='spa', config='--oem 3 --psm 6')
    return text

def extract_tables_from_pdf(pdf_path):
    """Extrae todas las tablas de un PDF y las guarda en un Excel con múltiples hojas"""
    tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
    return tables if tables else []

Guardamos el archivo en formato Word y Excel

In [103]:
def clean_text(text):
    """Limpia el texto eliminando caracteres no compatibles con XML"""
    return "".join(char if char.isprintable() else " " for char in text)

def save_text_as_docx(text, filename):
    """Guarda el texto extraído en un archivo Word."""
    text = clean_text(text)
    doc = Document()
    doc.add_paragraph(text)
    filename = get_unique_filename("output_files", filename)
    output_path = os.path.join("output_files", filename)
    doc.save(output_path)
    files.download(output_path)
    print(f"Archivo {output_path} listo para descarga.")

def save_tables_to_excel(tables, filename):
    """Guarda todas las tablas extraídas en un archivo Excel con múltiples hojas."""
    if tables:
        filename = get_unique_filename("output_files", filename)
        output_path = os.path.join("output_files", filename)
        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            for i, table in enumerate(tables):
                table.to_excel(writer, sheet_name=f"Tabla_{i+1}", index=False)
        files.download(output_path)
        print(f"Tablas guardadas en {output_path}, listas para descarga.")

Generamos la función para eliminar los archivos.

In [108]:
def cleanup():
    """Elimina todos los archivos de las carpetas de trabajo, ignorando directorios."""
    confirm = input("¿Eliminar archivos temporales? (s/n): ").strip().lower()
    if confirm == 's':
        for folder in ["upload_files", "output_files"]:
            for file in os.listdir(folder):
                file_path = os.path.join(folder, file)
                if os.path.isfile(file_path):  # Solo elimina archivos
                    os.remove(file_path)
        print("Archivos temporales eliminados.")

Flujo del programa principal para utilizar la Herramienta de OCR

In [109]:
# --- Flujo principal ---
archivo = upload_file()
if archivo:
    nombre_base = os.path.splitext(os.path.basename(archivo))[0]
    if archivo.endswith(".pdf"):
        calidad = evaluate_pdf_quality(archivo)
        print(f"Calidad estimada del PDF para OCR: {calidad:.2f}%")
        texto = extract_text_from_pdf(archivo)
        print("Extrayendo tablas...")
        tablas = extract_tables_from_pdf(archivo)
        save_tables_to_excel(tablas, f"{nombre_base}.xlsx")
    else:
        texto = extract_text_from_image(archivo)

    save_text_as_docx(texto, f"{nombre_base}.docx")
    cleanup()

    # Mejora con OpenAI (requiere API Key válida)
    # api_key = input("Introduce tu OpenAI API Key (opcional para mejora de texto): ")
    # if api_key:
        # texto = improve_text_with_openai(texto, api_key)
        # print("Texto mejorado con OpenAI.")



Saving Oferta PH Hornos V01.pdf to Oferta PH Hornos V01.pdf
Calidad estimada del PDF para OCR: 100.00%
Extrayendo tablas...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Tablas guardadas en output_files/Oferta PH Hornos V01.xlsx, listas para descarga.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivo output_files/Oferta PH Hornos V01.docx listo para descarga.
¿Eliminar archivos temporales? (s/n): s
Archivos temporales eliminados.
